## Evaluating FastText Embeddings
Very quick example for training and evaluating a FastText embedding.

**Note: the training set is very small!**

In [1]:
from datasets import load_dataset

# Load dataset from hugging face hub and make it a pandas dataframe
dataset = load_dataset("Paul/hatecheck", split="test").to_pandas()

# Only select hateful subset
dataset = dataset[dataset["label_gold"] == "hateful"]

dataset.head()

Found cached dataset csv (/Users/oskarvanderwal/.cache/huggingface/datasets/Paul___csv/Paul--hatecheck-de74109e77bfb6c3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


,functionality,case_id,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ
0,derog_neg_emote_h,1,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
1,derog_neg_emote_h,2,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
2,derog_neg_emote_h,3,I hate gay people.,hateful,gay people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
3,derog_neg_emote_h,4,I hate black people.,hateful,black people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].
4,derog_neg_emote_h,5,I hate disabled people.,hateful,disabled people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].


In [2]:
# Tokenize sentences
tokenized_dataset = [sentence.split(" ") for sentence in dataset["test_case"]]
tokenized_dataset[0]

['I', 'hate', 'women.', '']

In [3]:
from gensim.models import FastText
from gensim.test.utils import common_texts  # some example sentences

model = FastText(vector_size=4, window=3, min_count=1)  # instantiate

model.build_vocab(corpus_iterable=tokenized_dataset)

model.train(corpus_iterable=dataset["test_case"], total_examples=len(dataset["test_case"]), epochs=10)  # train

(851770, 1204860)

In [4]:
model.wv["man"]

array([ 0.05379008,  0.00660545,  0.08877474, -0.0180898 ], dtype=float32)

In [5]:
# Save the model
model.save("hatecheck_fasttext.model")

In [6]:
from biasbarometer.models import CharacterEmbeddingsModel

embedding = CharacterEmbeddingsModel("hatecheck_fasttext.model", lowercase=False).get_representation("embedding")

In [7]:
from biasbarometer.barometers import AutoBarometer

# Operationalize the barometer using Dutch word lists this time
barometer = AutoBarometer.from_spec("direction", wordpairs="../data/wordlists/man_vs_woman.csv", target="../data/wordlists/occupations.txt")

# Run the bias evaluation
barometer.evaluate(embedding)

barometer.results["score"]

0.020917229838202177

In [8]:
df = barometer.results["bias_df"]
df[df["category"]!="target"]

,word,score,category
0,his,0.179642,masculine
1,man,0.085892,masculine
2,male,0.066237,masculine
4,he,0.050377,masculine
6,john,0.033979,masculine
21,female,0.009900,feminine
27,woman,-0.001055,feminine
37,himself,-0.008759,masculine
40,mary,-0.011311,feminine
44,herself,-0.014709,feminine
